In [1]:
#pip install selenium

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import re

In [2]:
# Configuração para exibir colunas de largura total
pd.set_option('display.max_colwidth', None)

# Lista de times da Série A do Campeonato Brasileiro de 2023
serie_a_teams_2023 = ["Flamengo", "Palmeiras", "Atletico-Mineiro", "Fortaleza", "Bragantino", 
                      "Corinthians", "Internacional", "Athletico-Paranaense", "Santos", 
                      "Cuiaba", "Bahia", "Vasco-da-Gama", "Fluminense", "Sao-Paulo", "Cruzeiro", 
                      "America-MG", "Gremio", "Goias", "Botafogo-RJ", "Coritiba",
                      "Classico-Majestoso", "Classico-dos-Milhoes"]

# Inicializa o WebDriver do Selenium (certifique-se de ter o chromedriver ou geckodriver no seu PATH)
driver = webdriver.Chrome()

# URL da página
url = "https://fbref.com/en/comps/24/2023/schedule/2023-Serie-A-Scores-and-Fixtures"

# Abre a página no navegador
driver.get(url)

# Aguardar um tempo para garantir que a página seja totalmente carregada
time.sleep(2)

try:
    # Encontra todos os elementos <a> na página
    links = driver.find_elements(By.TAG_NAME, "a")
    
    # Lista para armazenar os links de matches
    matches_links = []
    
    # Itera sobre os elementos <a> e extrai os URLs dos matches
    for link in links:
        href = link.get_attribute('href')
        if href is not None and '/matches/' in href and 'Serie-A' in href and 'Serie-A1' not in href:

            for team in serie_a_teams_2023:
                if re.search(rf'/{re.escape(team)}-', href, re.IGNORECASE):
                    matches_links.append(href)
                    break
              
    # Cria um DataFrame pandas com os links dos matches
    df_links = pd.DataFrame(matches_links, columns=['Links'])

    # Remove os valores duplicados do DataFrame
    df_links.drop_duplicates(inplace=True)
    
    # Reset o índice
    df_links.reset_index(drop=True, inplace=True)
    
except Exception as e:
    print("Ocorreu um erro:", e)

finally:
    # Fecha o navegador
    driver.quit()

In [3]:
# Exibe o DataFrame
df_links.head(10)

,Links
0,https://fbref.com/en/matches/3f9c38cd/America-MG-Fluminense-April-15-2023-Serie-A
1,https://fbref.com/en/matches/b959f7a3/Palmeiras-Cuiaba-April-15-2023-Serie-A
2,https://fbref.com/en/matches/94647e41/Bragantino-Bahia-April-15-2023-Serie-A
3,https://fbref.com/en/matches/d5b8d3df/Botafogo-RJ-Sao-Paulo-April-15-2023-Serie-A
4,https://fbref.com/en/matches/d6629544/Athletico-Paranaense-Goias-April-15-2023-Serie-A
5,https://fbref.com/en/matches/f8e08c8f/Fortaleza-Internacional-April-15-2023-Serie-A
6,https://fbref.com/en/matches/1d8a4c36/Flamengo-Coritiba-April-16-2023-Serie-A
7,https://fbref.com/en/matches/9d3b8f29/Corinthians-Cruzeiro-April-16-2023-Serie-A
8,https://fbref.com/en/matches/e1f3d89b/Gremio-Santos-April-16-2023-Serie-A
9,https://fbref.com/en/matches/66c932af/Atletico-Mineiro-Vasco-da-Gama-April-15-2023-Serie-A


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Função para extrair dados de uma URL e preencher o DataFrame stats_df
def extract_data_from_url(url):
    
    # Reseta a variável
    matchweek_element = ""

    # Instanciando o navegador
    driver = webdriver.Chrome()

    # Carregar a página
    driver.get(url)

    # Aguardar um tempo para garantir que a página seja totalmente carregada
    time.sleep(2)

    # Encontrar o elemento que contém o nome dos times
    match_title_element = driver.find_element(By.XPATH, "//h1")

    # Extrair o texto do elemento
    match_title_text = match_title_element.text

    # Usar expressões regulares para extrair os nomes dos times
    #team_names = re.findall(r"([^vs]+) vs\. ([^\s]+(?: [^\s]+)*?) Match", match_title_text)
    # Usar expressões regulares para extrair os nomes dos times
    team_names = re.findall(r"([^\d-]+) vs\. ([^\d-]+).*?Match", match_title_text)

    # Encontrar o elemento que contém o número da Matchweek
    matchweek_element = driver.find_element(By.XPATH, "//div[contains(text(), 'Matchweek')]")

    # Extrair o número da Matchweek usando expressões regulares
    matchweek_number = re.search(r"Matchweek (\d+)", matchweek_element.text).group(1)

    # Encontrar todas as tabelas na página
    tables = driver.find_elements(By.TAG_NAME, "table")

    # Inicializar uma lista para armazenar os DataFrames
    dfs = []

    # Iterar sobre as tabelas 9 e 10
    for i in range(9, 11):
        # Obter o conteúdo HTML da tabela
        html_table = tables[i].get_attribute("outerHTML")

        # Parsear o conteúdo HTML com BeautifulSoup
        soup = BeautifulSoup(html_table, 'html.parser')

        # Converter a tabela para um DataFrame do Pandas e adicionar à lista
        df = pd.read_html(str(soup))[0]
        dfs.append(df)

    # Concatenar os DataFrames horizontalmente
    lineup_df = pd.concat(dfs, axis=1)

    # Selecionar apenas as 11 primeiras linhas do DataFrame combinado
    lineup_df = lineup_df.iloc[:11]

    # Remover as colunas 0 e 2 do DataFrame
    lineup_df = lineup_df.drop(lineup_df.columns[[0, 2]], axis=1)

    # Renomear a primeira coluna para 'Player Home'
    lineup_df = lineup_df.rename(columns={lineup_df.columns[0]: 'Player Home'})
    lineup_df = lineup_df.rename(columns={lineup_df.columns[1]: 'Player Away'})

    # Inicializar uma lista para armazenar os DataFrames
    dfs = []
    numberRow = 0

    # Iterar sobre as tabelas 12 a 18
    for i in range(12, 19):
        # Obter o conteúdo HTML da tabela
        html_table = tables[i].get_attribute("outerHTML")

        # Parsear o conteúdo HTML com BeautifulSoup
        soup = BeautifulSoup(html_table, 'html.parser')

        # Converter a tabela para um DataFrame do Pandas e adicionar à lista
        df = pd.read_html(str(soup))[0]

        # Remover a primeira linha de títulos
        df.columns = df.columns.droplevel(level=0)

        #Conta o número de linhas do dataframe
        if(i == 12):
            numberRow = len(df) - 1
        #Se chegou na última tabela
        elif(i == 18):

            # Inserir linhas vazias na posição 0 com valores zero
            for _ in range(numberRow - len(df)):
                # Insere uma nova linha vazia na posição 0
                df.loc[-1] = [0] * len(df.columns)
                df.index = df.index + 1  # Atualiza os índices
                df = df.sort_index()  # Ordena os índices

            # Redefinir os índices das linhas
            df = df.reset_index(drop=True)

        # Adicionar o DataFrame à lista
        dfs.append(df)

    # Concatenar os DataFrames lado a lado
    combined_df = pd.concat(dfs, axis=1)

    # Remover a primeira e a última linha do DataFrame combinado
    combined_df = combined_df.iloc[0:-1]

    # Renomear a coluna "#"
    combined_df = combined_df.rename(columns={"#": "Number"})

    # Renomear a coluna
    combined_df.columns.values[[31, 59, 80, 102, 130, 152]] = 'Drop'
    combined_df.columns.values[[32, 60, 81, 103, 131]] = 'Drop'
    combined_df.columns.values[[33, 61, 82, 104, 132, 153]] = 'Drop'
    combined_df.columns.values[[34, 62, 83, 105, 133]] = 'Drop'
    combined_df.columns.values[[35, 63, 84, 106, 134, 154]] = 'Drop'
    combined_df.columns.values[[36, 64, 85, 107, 135, 155]] = 'Drop'

    # Excluir a coluna 'Drop'
    combined_df = combined_df.drop(columns='Drop')

    # Incluir as colunas "Game", "Team" e "Holder" preenchendo com None
    combined_df.insert(0, "Game", matchweek_number)
    combined_df.insert(1, "Team", team_names[0][0])

    # Verifique se o jogador está presente em lineup_df e armazene os resultados em uma lista
    is_present = []
    for player in combined_df['Player'].values:
        if (player in lineup_df['Player Home'].values):
            is_present.append('Y')
        else:
            is_present.append('N')
    
    combined_df.insert(2, "Holder", is_present)

    stats_df = combined_df

    # Inicializar uma lista para armazenar os DataFrames
    dfs = []
    combined_df = []
    numberRow = 0

    # Iterar sobre as tabelas 19 a 25
    for i in range(19, 26):
        # Obter o conteúdo HTML da tabela
        html_table = tables[i].get_attribute("outerHTML")

        # Parsear o conteúdo HTML com BeautifulSoup
        soup = BeautifulSoup(html_table, 'html.parser')

        # Converter a tabela para um DataFrame do Pandas e adicionar à lista
        df = pd.read_html(str(soup))[0]

        # Remover a primeira linha de títulos
        df.columns = df.columns.droplevel(level=0)

        #Conta o número de linhas do dataframe
        if(i == 19):
            numberRow = len(df) - 1
        #Se chegou na última tabela
        elif(i == 25):

            # Inserir linhas vazias na posição 0 com valores zero
            for _ in range(numberRow - len(df)):
                # Insere uma nova linha vazia na posição 0
                df.loc[-1] = [0] * len(df.columns)
                df.index = df.index + 1  # Atualiza os índices
                df = df.sort_index()  # Ordena os índices

            # Redefinir os índices das linhas
            df = df.reset_index(drop=True)

        # Adicionar o DataFrame à lista
        dfs.append(df)

    # Concatenar os DataFrames lado a lado
    combined_df = pd.concat(dfs, axis=1)

    # Remover a primeira e a última linha do DataFrame combinado
    combined_df = combined_df.iloc[0:-1]

    # Renomear a coluna "#"
    combined_df = combined_df.rename(columns={"#": "Number"})

    # Renomear a coluna
    combined_df.columns.values[[31, 59, 80, 102, 130, 152]] = 'Drop'
    combined_df.columns.values[[32, 60, 81, 103, 131]] = 'Drop'
    combined_df.columns.values[[33, 61, 82, 104, 132, 153]] = 'Drop'
    combined_df.columns.values[[34, 62, 83, 105, 133]] = 'Drop'
    combined_df.columns.values[[35, 63, 84, 106, 134, 154]] = 'Drop'
    combined_df.columns.values[[36, 64, 85, 107, 135, 155]] = 'Drop'

    # Excluir a coluna 'Drop'
    combined_df = combined_df.drop(columns='Drop')

    # Incluir as colunas "Game", "Team" e "Holder" preenchendo com None
    combined_df.insert(0, "Game", matchweek_number)
    combined_df.insert(1, "Team", team_names[0][1])

    # Verifique se o jogador está presente em lineup_df e armazene os resultados em uma lista
    is_present = []
    for player in combined_df['Player'].values:
        if (player in lineup_df['Player Away'].values):
            is_present.append('Y')
        else:
            is_present.append('N')
    
    combined_df.insert(2, "Holder", is_present)

    # Concatenar os DataFrames
    stats_df = pd.concat([stats_df, combined_df], ignore_index=True)

    # Execute um script JavaScript para obter o texto dos elementos de score
    scores = driver.execute_script("""
        scores = document.querySelectorAll('div.scores div.score');
        return [scores[0].textContent, scores[1].textContent];
    """)

    # Número de gols por time
    scoreHome = scores[0]
    scoreAway = scores[1]

    table = tables[11]  # O índice 11 representa a 12ª tabela (o índice começa em 0)
    # Use BeautifulSoup para analisar o HTML da tabela
    soup = BeautifulSoup(table.get_attribute('outerHTML'), 'html.parser')

    # Extrai os dados da tabela
    data = []
    for row in soup.find_all('tr'):
        row_data = []
        for cell in row.find_all(['th', 'td']):
            row_data.append(cell.text.strip())
        if row_data:
            data.append(row_data)

    # Inicialize as variáveis
    possessionHome = None
    possessionAway = None
    passingAccuracyHome = None
    passingAccuracyAway = None
    shotsOnTargetHome = None
    shotsOnTargetAway = None
    savesHome = None
    savesAway = None

    # Itere sobre os dados extraídos da tabela
    for i in range(len(data)):
        if 'Possession' in data[i][0]:
            if len(data[i+1]) > 1:
                possessionHome_match = re.search(r'\d+%', data[i+1][0])
                possessionAway_match = re.search(r'\d+%', data[i+1][1])
                if possessionHome_match:
                    possessionHome = possessionHome_match.group().replace('%', '')
                if possessionAway_match:
                    possessionAway = possessionAway_match.group().replace('%', '')
        elif 'Passing Accuracy' in data[i][0]:
            if len(data[i+1]) > 1:
                passingAccuracyHome_match = re.search(r'\d+%', data[i+1][0])
                passingAccuracyAway_match = re.search(r'\d+%', data[i+1][1])
                if passingAccuracyHome_match:
                    passingAccuracyHome = passingAccuracyHome_match.group().replace('%', '')
                if passingAccuracyAway_match:
                    passingAccuracyAway = passingAccuracyAway_match.group().replace('%', '')
        elif 'Shots on Target' in data[i][0]:
            if len(data[i+1]) > 1:
                shotsOnTargetHome_match = re.search(r'\d+%', data[i+1][0])
                shotsOnTargetAway_match = re.search(r'\d+%', data[i+1][1])
                if shotsOnTargetHome_match:
                    shotsOnTargetHome = shotsOnTargetHome_match.group().replace('%', '')
                if shotsOnTargetAway_match:
                    shotsOnTargetAway = shotsOnTargetAway_match.group().replace('%', '')
        elif 'Saves' in data[i][0]:
            if len(data[i+1]) > 1:
                savesHome_match = re.search(r'\d+%', data[i+1][0])
                savesAway_match = re.search(r'\d+%', data[i+1][1])
                if savesHome_match:
                    savesHome = savesHome_match.group().replace('%', '')
                if savesAway_match:
                    savesAway = savesAway_match.group().replace('%', '')

    # Criar os dados para o DataFrame
    data = {
        'Game': [matchweek_number],
        'Home': [team_names[0][0]],
        'HomeGoal': [scoreHome],
        'HomePossession': [possessionHome],
        'HomePassingAccuracy': [passingAccuracyHome],
        'HomeShotsTarget': [shotsOnTargetHome],
        'HomeSaves': [savesHome],
        'Away': [team_names[0][1]],
        'AwayGoal': [scoreAway],
        'AwayPossession': [possessionAway],
        'AwayPassingAccuracy': [passingAccuracyAway],
        'AwayShotsTarget': [shotsOnTargetAway],
        'AwaySaves': [savesAway],
    }

    # Criar o DataFrame
    results_df = pd.DataFrame(data)

    # Fechar o navegador
    driver.quit()

    return stats_df, results_df

In [24]:
# Inicializar uma lista para armazenar os DataFrames de cada URL
stats_df = []
results_df = []

# Convertendo a série de URLs em um array do NumPy
links_array = df_links['Links'].to_numpy()

# Dividindo o array em quatro partes
links_parts = np.array_split(links_array, 4)

In [25]:
# Iterando sobre a primeira parte do array
for url in links_parts[0]:
    # Extrair dados da URL atual e adicionar ao DataFrame
    dfS, dfR = extract_data_from_url(url)
    stats_df.append(dfS)
    results_df.append(dfR)

In [26]:
# Iterando sobre a primeira parte do array
for url in links_parts[1]:
    # Extrair dados da URL atual e adicionar ao DataFrame
    dfS, dfR = extract_data_from_url(url)
    stats_df.append(dfS)
    results_df.append(dfR)

In [27]:
# Iterando sobre a primeira parte do array
for url in links_parts[2]:
    # Extrair dados da URL atual e adicionar ao DataFrame
    dfS, dfR = extract_data_from_url(url)
    stats_df.append(dfS)
    results_df.append(dfR)

In [28]:
# Iterando sobre a primeira parte do array
for url in links_parts[3]:
    # Extrair dados da URL atual e adicionar ao DataFrame
    dfS, dfR = extract_data_from_url(url)
    stats_df.append(dfS)
    results_df.append(dfR)

In [29]:
# Concatenar os DataFrames
stats_df = pd.concat(stats_df, ignore_index=True)
results_df = pd.concat(results_df, ignore_index=True)

In [30]:
stats_df.head(3)

,Game,Team,Holder,Player,Number,Nation,Pos,Age,Min,Gls,...,Launch%,AvgLen,Att,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,AvgDist
0,1,América (MG),Y,Henrique Almeida,19.0,br BRA,FW,31-323,62,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,América (MG),N,Wellington Paulista,9.0,br BRA,FW,38-358,28,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,América (MG),Y,Felipe Azevedo,11.0,br BRA,LW,36-095,68,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
results_df.head(3)

,Game,Home,HomeGoal,HomePossession,HomePassingAccuracy,HomeShotsTarget,HomeSaves,Away,AwayGoal,AwayPossession,AwayPassingAccuracy,AwayShotsTarget,AwaySaves
0,1,América (MG),0,46,74,12,57,Fluminense,3,54,83,50,100
1,1,Palmeiras,2,52,82,29,75,Cuiabá,1,48,78,33,60
2,1,Bragantino,2,55,75,54,66,Bahia,1,45,69,33,71


In [32]:
# Salvar o DataFrame final em um arquivo Excel
stats_df.to_excel("Brazilian_Players_Statistics.xlsx", index=False)

In [33]:
# Salvar o DataFrame final em um arquivo Excel
results_df.to_excel("Brazilian_Teams_Statistics.xlsx", index=False)